In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#TF stuff
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

#XGBoost
from xgboost import XGBClassifier

#Scikit
from sklearn.metrics import log_loss
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [ ]:
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
print(train_features.head())
print(train_targets.head())
print(train_features.shape)
print(train_targets.shape)

In [ ]:
train_features_enc = pd.get_dummies(train_features, columns=['cp_type', 'cp_dose'], drop_first=True)
print(train_features_enc.head())

In [ ]:
X = train_features_enc.iloc[:,1:].to_numpy()
y = train_targets.iloc[:,1:].to_numpy() 

In [ ]:
def l3_model(input_shape, no_classes, lr):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(128, activation='sigmoid')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='sigmoid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='sigmoid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(no_classes, activation='sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate = lr), metrics=['binary_crossentropy'])
    return model

In [ ]:
nnclfl3_r = l3_model((875,),206,0.0005) 
hist_l3_r = nnclfl3_r.fit(X, y, batch_size=512, epochs=50, verbose=0)
preds_l3_r = nnclfl3_r.predict(X)

In [ ]:
#Getting Data
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
test_features_enc = pd.get_dummies(test_features, columns=['cp_type', 'cp_dose'], drop_first=True)

X_test = test_features_enc.iloc[:,1:].to_numpy() 

In [ ]:
preds_l3_r = nnclfl3_r.predict(X_test) # list of preds per class
preds_ensemble_test = preds_l3_r

In [ ]:
#Submitting the results
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

preds_ensemble_test_df = pd.DataFrame(preds_ensemble_test,
                              columns = sample_submission.columns[1:])

preds_ensemble_test_df.insert(0, sample_submission.columns[0],
                     test_features[sample_submission.columns[0]])

preds_ensemble_test_df.to_csv('submission.csv', index=False)